In [1]:
# Dependencies
import json
import requests 
from pprint import pprint
import pandas as pd
from pandas import json_normalize
import urllib
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
from datetime import datetime, date
import schedule 
import time

In [15]:
# Specify the URL
url = "https://gbfs.citibikenyc.com/gbfs/es/station_status.json"

# Make request and store response
response = requests.get(url)

# Print status code
print(response.status_code)

In [3]:
r = requests.get('https://gbfs.citibikenyc.com/gbfs/en/station_status.json')
df = json_normalize(r.json()['data']['stations'])
df.head()

,num_bikes_available,eightd_has_available_keys,station_id,last_reported,num_bikes_disabled,is_returning,legacy_id,is_installed,station_status,is_renting,...,num_docks_disabled,num_ebikes_available,eightd_active_station_services,valet.valet_revision,valet.station_id,valet.region,valet.active,valet.off_dock_capacity,valet.dock_blocked_count,valet.off_dock_count
0,11,False,72,1596556984,2,1,72,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24,False,79,1596558392,0,1,79,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,False,82,1596554480,0,1,82,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,False,83,1596558152,1,1,83,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,False,116,1596558090,2,1,116,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df2 = df.rename(columns={"num_bikes_available":"bikes_available","num_bikes_disabled":"bikes_disabled","num_docks_available":"docks_available","num_docks_disabled":"docks_disabled","last_reported":"time_reported"})

In [5]:
df2.head()

,bikes_available,eightd_has_available_keys,station_id,time_reported,bikes_disabled,is_returning,legacy_id,is_installed,station_status,is_renting,...,docks_disabled,num_ebikes_available,eightd_active_station_services,valet.valet_revision,valet.station_id,valet.region,valet.active,valet.off_dock_capacity,valet.dock_blocked_count,valet.off_dock_count
0,11,False,72,1596556984,2,1,72,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24,False,79,1596558392,0,1,79,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,False,82,1596554480,0,1,82,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35,False,83,1596558152,1,1,83,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,False,116,1596558090,2,1,116,1,active,1,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
bike_df = df2[["station_id","bikes_available","bikes_disabled","docks_available","docks_disabled","time_reported"]]

In [7]:
bike_df.head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,11,2,42,0,1596556984
1,79,24,0,9,0,1596558392
2,82,10,0,17,0,1596554480
3,83,35,1,26,0,1596558152
4,116,9,2,39,0,1596558090


In [8]:

last_reported = datetime.fromtimestamp(bike_df.loc[0,"time_reported"])

In [9]:
last_reported

datetime.datetime(2020, 8, 4, 9, 3, 4)

In [10]:
def epoch_to_readable(xepoch):
    
    return datetime.fromtimestamp(xepoch)

bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )

<ipython-input-10-5919d25c1233>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bike_df['time_reported'] = bike_df.apply(lambda x: epoch_to_readable(x['time_reported']), axis = 1 )


In [11]:
bike_df.head()

,station_id,bikes_available,bikes_disabled,docks_available,docks_disabled,time_reported
0,72,11,2,42,0,2020-08-04 09:03:04
1,79,24,0,9,0,2020-08-04 09:26:32
2,82,10,0,17,0,2020-08-04 08:21:20
3,83,35,1,26,0,2020-08-04 09:22:32
4,116,9,2,39,0,2020-08-04 09:21:30


In [12]:
db_connection_string = "postgres:jaigurudev@localhost:5432/FinalProject"
engine = create_engine(f'postgresql://{db_connection_string}')

In [13]:
engine.table_names()

['bikes']

In [14]:
bike_df.to_sql(name = 'bikes',con = engine , if_exists = 'append',index = False)

In [ ]:
pd.read_sql_query('select * from bikes', con=engine).head()